# Messungen mit JupyterLab auf dem STEMlab

Wie Sie Jupyter Notebook starten und ein neues Projekt erstellen, ist in [Figure](23_fig_05.html#23_fig_05) dargestellt. 

![Erstellen eines neuen Jupyter Notbooks.](../images/Activity_19_Fig_07.png)

Wenn Sie erfolgreich ein neues Jupyter Notebook erstellt haben, kopieren Sie den untenstehenden Code und führen Sie ihn aus.

Der unten stehende Code erzeugt das gleiche Signal wie in [Figure](23_fig_04.html#23_fig_04), aber er plotet dieses in einem XY-Diagramm.

Für die Messung der IV-Kurve ist ein XY-Plot erforderlich, wobei die x-Achse die Diodenspannung $IN_2$ und die y-Achse
den Diodenstrom (Drainstrom) ) $I_D = (IN_1 - IN_2)/R_3$ darstellen. 

Kopieren Sie den unten stehenden Code in Zelle 1:

In [ ]:
# Import libraries
from redpitaya.overlay.mercury import mercury as overlay
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d, LinearAxis, LabelSet, Label
from bokeh.plotting import figure, output_file, show
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
import numpy as np

# Initialize fpga modules
fpga = overlay()
gen0 = fpga.gen(0)
osc = [fpga.osc(ch, 1.0) for ch in range(fpga._MNO)]

# Configure OUT1 generator channel
gen0.amplitude = 0.45
gen0.offset = -0.45
gen0.waveform = gen0.sawtooth(0.5)
gen0.frequency = 1000
gen0.start()
gen0.enable = True
gen0.trigger()

# Resistor value
R3 = 1000

# Configure IN1 and IN2 oscilloscope input channels
for ch in osc:
    ch.filter_bypass = True

    # data rate decimation
    ch.decimation = 10

    # trigger timing [sample periods]
    N = ch.buffer_size
    ch.trigger_pre = 0
    ch.trigger_post = N

    # osc0 is controlling both channels
    ch.sync_src = fpga.sync_src["osc0"]
    ch.trig_src = fpga.trig_src["osc0"]

    # trigger level [V], edge ['neg', 'pos'] and holdoff time [sample periods]
    ch.level = 0.01
    ch.edge = 'pos'
    ch.holdoff = 0

    
# Initialize diode current and voltage
V = np.zeros(N)
I = np.zeros(N)

# Plotting
hover = HoverTool(mode='vline', tooltips=[("V", "@x"), ("I", "@y")])
tools = "wheel_zoom, box_zoom, reset,pan"
p = figure(plot_height=500,
           plot_width=900,
           title="XY Plot der NMOS Transistor IV Charakteristik",
           toolbar_location="right",
           tools=(tools, hover))
p.xaxis.axis_label = 'Spannung in V'
p.yaxis.axis_label = 'Strom in mA'
r = p.line(V, I, line_width=1, line_alpha=0.7, color="blue")

# get and explicit handle to update the next show cell
target = show(p, notebook_handle=True)

Erstelle eine neue Zelle (Einfügen -> Zelle darunter) und kopieren Sie den unten stehenden Code.


In [ ]:
# Measuring I, V and re-plotting
while True:
    # reset and start
    osc[0].reset()
    osc[0].start()

    # wait for data
    while (osc[0].status_run()):
        pass

    V0 = osc[0].data(N-100)*10  # IN1 signal
    V1 = osc[1].data(N-100)*10  # IN2 signal
    I = ((V0-V1)/R3)*1E3        # 1E3 convert to mA
    r.data_source.data['x'] = V0
    r.data_source.data['y'] = I
    push_notebook(handle=target)

Führen Sie Zelle 1 und Zelle 2 aus. Hinweis Zelle 2 ist eine Hauptschleife für die Erfassung und Neuaufnahme. Wenn Sie
die Erfassung stoppen, führen Sie einfach nur Zelle 2 aus, um die Messungen erneut zu starten.

Nach dem Ausführen der Jupyter Notebook Zelle sollten Sie die IV-Charakteristik der Diode erhalten, wie in [Figure](23_fig_06.html#23_fig_06) dargestellt.

![nMOS IV-Kennlinie gemessen mit Jupyter Notebook.](../images/Activity_23_Fig_06.png)